# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [14]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2023-06-01	In or Out? Fixing ImageNet Out-of-distribution detection	ICML	We show that previously used datasets for evaluating OOD detection on ImageNet are severly flawed, because they contain a significant fraction of ID samples. We present NINCO, a novel hand-cleaned dataset as a solution.	J. Bitterwolf*, M. Müller* and Matthias Hein (*equal contribution) (2023) “In or out? Fixing ImageNet Out-of-distribution detection” <i> ICML 2023</i>	NINCO	https://arxiv.org/abs/2306.00826
2023-02-14	A modern look at the relationship between sharpness and generaliization	ICML	In a large-scale study we observe that sharpness does not correlate well with generalization but rather with some training parameters.	Andriushchenko M., Croce F., Müller M., Hein M.,  and Flammarion N. (2023)  “A modern look at the relationship between sharpness and generaliization” <i> ICML 2023</i>	sharpness	https://arxiv.org/abs/2302.07011
2023-06-07	Normalization L

## Import pandas

We are using the very handy pandas library for dataframes.

In [15]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [16]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2023-06-01,In or Out? Fixing ImageNet Out-of-distribution...,ICML,We show that previously used datasets for eval...,"J. Bitterwolf*, M. Müller* and Matthias Hein (...",NINCO,https://arxiv.org/abs/2306.00826
1,2023-02-14,A modern look at the relationship between shar...,ICML,In a large-scale study we observe that sharpne...,"Andriushchenko M., Croce F., Müller M., Hein M...",sharpness,https://arxiv.org/abs/2302.07011
2,2023-06-07,Normalization Layers Are All That Sharpness-Aw...,NeurIPS,We show that perturbing only the affine normal...,"Maximilian Müller, Tiffany Vlaar, David Rolnic...",SAMON,https://arxiv.org/abs/2306.04226
3,2022-05-02,Leveraging Stochastic Predictions of Bayesian ...,NeurIPS,We investigate uncertainty estimation and mult...,"Maximilian Mueller, Robin Greif, Frank Jenko, ...",BNN-Fluid,https://arxiv.org/abs/2205.01222
4,2021-09-11,Physics-based Deep Learning,world wide web,This document contains a practical and compreh...,"Nils Thuerey, Philipp Holl, Maximilian Mueller...",PBDL,https://physicsbaseddeeplearning. org
5,2018-12-10,Fast evaluation of the current driven by elect...,Physics of Plasmas,"In this paper, a procedure for the evaluation ...","E Poli, M Müller, H Zohm, M Kovari (2018) “Fas...",Eccd-plasma,https://pure.mpg.de/rest/items/item_3018972/co...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [17]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [18]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [19]:
!ls ../_publications/

2018-12-10-Eccd-plasma.md  2022-05-02-BNN-Fluid.md  2023-06-01-NINCO.md
2021-09-11-PBDL.md	   2023-02-14-sharpness.md  2023-06-07-SAMON.md


In [20]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
